
# Build a basic ME model

We will try to build an ME model from the NC_000913.2 Genbank file, the iJO1366 M model, and the complex reconstruction from iJL1650-ME

In [1]:
# python imports
import re
import json
from os.path import join

# third party imports
import pandas
import escher
import cobra.test

# ecoli me
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *

from minime import *
from minime.util.building import build_reactions_from_genbank, add_transcription_reaction
from minime.util.building import add_ecoli_M_model_content, add_ribosomes, add_RNA_polymerase
from minime.util.building import add_TUs_and_translation, add_generic_RNase
from minime.solve.algorithms import binary_search, fva, solve_at_growth_rate
from minime.solve.symbolic import compile_expressions

In [2]:
met_info = pandas.read_csv(join(ecoli_files_dir, "metabolites.txt"),
                           delimiter="\t", header=None, index_col=0,
                           names=["id", "name", "formula", "compartment",
                                  "data_source"])
rxn_info = get_reaction_info_frame()
rxn_dict = get_reaction_matrix_dict()
sources_sinks_info = pandas.read_csv(join(ecoli_files_dir, "reaction_matrix_sources_and_sinks.txt"), delimiter="\t",
                                     header=None, names=["rxn_id", "met_id", "compartment", "stoic"], index_col=1)

source_amounts = pandas.read_csv(join(ecoli_files_dir, "exchange_bounds.txt"),
                                 delimiter="\t", index_col=0, names=["met_id", "amount"])

## Begin by loading metabolites

In [3]:
me = MEmodel('iJO1366-ME')
add_ecoli_M_model_content(me, 'iJO1366', met_info, rxn_info, rxn_dict, 
                          sources_sinks_info, source_amounts,generic_ions=True)

Assigned tl to e
Assigned hemed to e
Assigned pqq to e
Assigned dpm to e
Assigned 23bpg to e
Assigned tqn to e


## Build Metabolic reactions

Also make a dummy reaction

In [4]:
me.reactions.EX_pqq_e.lower_bound = -1000

In [5]:
dummy = StoichiometricData("dummy_reaction", me)
dummy.lower_bound = 0
dummy.upper_bound = 1000
dummy._stoichiometry = {}

## Build ribosome and RNA Polymerase

In [6]:
gb_filename = join(ecoli_files_dir,'NC_000913.2.gb')                                                                                    
TU_df = pandas.read_csv(join(ecoli_files_dir,'TUs_from_ecocyc.txt'), delimiter="\t", index_col=0)            

TU_pieces = add_TUs_and_translation(me, gb_filename, TU_frame=TU_df)

/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:217 UserWarning: ribosome not found
/home/sbrg-cjlloyd/minime/minime/core/MEReactions.py:153 UserWarning: RNA Polymerase not found


TODO deal with selenocystine
unknown metabolite 'RNase_T_dim_mod_4:mg2' created
unknown metabolite 'generic_RNase' created
unknown metabolite 'RNase_BN_dim_mod_2:zn2' created
unknown metabolite 'Rnd_mono_mod_5:mg2' created
unknown metabolite 'Rnb_mono_mod_1:mg2' created
unknown metabolite 'Rph_mono_mod_mg2' created


In [7]:
add_ribosomes(me)

Created <Metabolite InfA_mono at 0x7f55f56ba0d0> in <ComplexFormation formation_ribosome at 0x7f55f34a3150>
Created <Metabolite RimM_mono at 0x7f55f2fe1150> in <ComplexFormation formation_ribosome at 0x7f55f34a3150>
Created <Metabolite Tig_mono at 0x7f55f2fe1210> in <ComplexFormation formation_ribosome at 0x7f55f34a3150>
Created <Metabolite InfC_mono at 0x7f55f2fe11d0> in <ComplexFormation formation_ribosome at 0x7f55f34a3150>
Created <Metabolite InfB_mono at 0x7f55f2fe1190> in <ComplexFormation formation_ribosome at 0x7f55f34a3150>
Created <Metabolite RbfA_mono at 0x7f55f2fe1250> in <ComplexFormation formation_ribosome at 0x7f55f34a3150>
Created <Metabolite Era_dim at 0x7f55f2fe12d0> in <ComplexFormation formation_ribosome at 0x7f55f34a3150>


In [8]:
add_RNA_polymerase(me)

Created <Metabolite hRNAP at 0x7f55f2fe1450> in <ComplexFormation formation_RNA_Polymerase at 0x7f55f2fe1410>


In [12]:
for cplx in me.complex_data:
    for protein in cplx.stoichiometry.keys():
        try:
            if len(me.metabolites.get_by_id(protein).reactions) == 1:
                print protein
        except:
            print cplx.parent_reactions, protein, me.metabolites.get_by_id(protein.replace('protein','RNA')).RNA_type

RpsU_mono
InfA_mono
RplA_mono
RplB_mono
RpsM_mono
RplX_mono
RplS_mono
RplN_mono
RpsF_mono
RplI_mono
RpsJ_mono
RplD_mono
RpsL_mono
RpsO_mono
RplU_mono
Sra_mono
RplP_mono
RpsB_mono
RplK_mono
RpsD_mono
RplV_mono
RpmH_mono
Tig_mono
RplC_mono
RplE_mono
RplF_mono
RpsA_mono
RpsP_mono
RpmJ_mono
RplQ_mono
RpsH_mono
RplT_mono
RpsQ_mono
RpsC_mono
rpL7/12_mod_1:acetyl
RpsG_mono
RpsI_mono
RpmF_mono
RpmE_mono
InfC_mono
RplR_mono
RpsS_mono
RpmI_mono
RpmG_mono
InfB_mono
RpmC_mono
RplJ_mono
RplW_mono
RplY_mono
RplO_mono
RpmD_mono
RpmB_mono
RpmA_mono
RpsN_mono
RplM_mono
RpsR_mono
RpsE_mono
RpsT_mono
RpsK_mono
hRNAP


### Add Transcription/Translation from the genbank file

Add a dummy protein in as well

In [13]:
dna_sequence = "ATG" + "TTT" * 12 + "TAT"*12+ "ACG"*12 + "GAT" *12 + "AGT"*12+ "TAA"
add_transcription_reaction(me, "dummy", {"dummy"}, dna_sequence)
me.add_metabolites(TranslatedGene("protein_" + "dummy"))
translation = TranslationReaction("translation_" + "dummy")
me.add_reaction(translation)
translation.translation_data = TranslationData("dummy", me, "RNA_dummy", "protein_dummy")
translation.translation_data.compute_sequence_from_DNA(dna_sequence)
translation.update()

complex_data = ComplexData("CPLX_dummy", me)
complex_data.stoichiometry = {}
complex_data.stoichiometry["protein_" + "dummy"] = 1
complex_data.create_complex_formation()

## Associate the tRNA synthetases

The tRNA charging reactions were automatically added when loading the genome from the genbank file. However, the charging reactions still need to be made aware of the tRNA synthetases which are responsible.

In [14]:
with open(join(ecoli_files_dir, "amino_acid_tRNA_synthetase.json"), "rb") as infile:
    aa_synthetase_dict = json.load(infile)
for data in me.tRNA_data:
    data.synthetase = str(aa_synthetase_dict[data.amino_acid])

### Add in complex Formation with modifications

In [15]:
# ME_complex_dict is a dict of {'complex_id': ['bnum(count)']}
ME_complex_dict = get_complex_to_bnum_dict()
# some entries in the file need to be renamed
renames = {"MnmE_": "b3706", "MnmG_": "b3741", "YheM_": "b3344", "YheL_": "b3343", "YheN_": "b3345"}
rna_components = {"b3123"}  # component id should have RNA_ instead of protein_

for cplx, value in ME_complex_dict.iteritems():
    complex_data = ComplexData(cplx, me)
    for gene in value:
        stoichiometry = gene[6]
        bnum = gene[0:5]
        comp_id = "RNA_" + bnum if bnum in rna_components \
            else "protein_" + renames.get(bnum, bnum)
        try:
            complex_data.stoichiometry[comp_id] = float(stoichiometry)
        except:
            complex_data.stoichiometry[comp_id] = float(1)

In [16]:
# {modified_complex_id: ['unmodified_complex_id', {component_id: stoic}]
modification_dict = get_protein_modification_dict(generic=True)
for mod_cplx_id, mod_complex_info in iteritems(modification_dict):
    unmod_cplx_id, mods = mod_complex_info
    unmod_cplx = me.complex_data.get_by_id(unmod_cplx_id)
    cplx = ComplexData(mod_cplx_id, me)
    cplx.stoichiometry = unmod_cplx.stoichiometry
    cplx.translocation = unmod_cplx.translocation
    cplx.chaperones = unmod_cplx.chaperones
    if len(set(mod_cplx_id.split("_mod_")[1:])) == len(mods):
        for mod_comp, mod_count in iteritems(mods):
            mod_id = "mod_" + mod_comp
            try:
                mod = me.modification_data.get_by_id(mod_id)
            except:
                mod = ModificationData(mod_id, me)
                mod.stoichiometry = {mod_comp: -1}
            cplx.modifications[mod_id] = -mod_count
    else:
        print "TODO:", mod_cplx_id

TODO: hRNAP_mod_1:generic_divalent_mod_2:generic_divalent
TODO: GltX_mono_mod_generic_divalent_mod_1:generic_divalent


Some modifications are enzyme-catalyzed

TODO: Fe-S enzyme-catalyzed modifications

In [17]:
# two different reactiond can add a lipoate modification.
# We create a separate ModificationData for each one
lipo = me.modification_data.get_by_id("mod_lipo_c")
alt_lipo = ModificationData("mod_lipo_c_alt", me)
alt_lipo.stoichiometry = lipo.stoichiometry

lipo.stoichiometry = {"lipoamp_c": -1, "amp_c": 1}
lipo.enzyme = 'EG11796-MONOMER'
lipo.keff = 65.

alt_lipo.stoichiometry = {'EG50003-MONOMER_mod_pan4p_mod_lipo':-1,
                          'EG50003-MONOMER_mod_pan4p':1,
                          'h_c':-1,}
alt_lipo.enzyme = 'EG11591-MONOMER'
alt_lipo.keff = 65.

for cplx_data in lipo.get_complex_data():
    alt_cplx_data = ComplexData(cplx_data.id + "alt", me)
    alt_cplx_data.complex_id = cplx_data.complex_id
    alt_cplx_data.translocation = cplx_data.translocation
    alt_cplx_data.chaperones = cplx_data.chaperones
    alt_cplx_data.modifications = cplx_data.modifications
    alt_cplx_data.modifications[alt_lipo.id] = \
        alt_cplx_data.modifications.pop(lipo.id)

In [18]:
# todo bmocogdp_c mods
mod_catalysts = {'CPLX0-1762':'G6712-MONOMER', # FE-S modification
                 'TMAOREDUCTI-CPLX':'EG12195-MONOMER',
                 'DIMESULFREDUCT-CPLX':'G6849-MONOMER',
                 'NITRATREDUCTA-CPLX':'NARJ-MONOMER', 
                 'NITRATREDUCTZ-CPLX':'NARW-MONOMER',
                 'NAP-CPLX':'NAPD-MONOMER',
                 'NAPAB-CPLX_NAPC-MONOMER':'NAPD-MONOMER'}

In [19]:
target_list = ['4fe4s_c', 'LI_c', '2fe2s_c', '3fe4s_c',
               'NiFeCoCN2_c', 'acetyl_c',
               'RNase_m5','RNase_m16','RNase_m23'] # RNAses are gaps in model

for met_id in target_list:
    r = cobra.Reaction("EX_" + met_id)
    me.add_reaction(r)
    r.reaction = met_id + " <=> "

unknown metabolite '4fe4s_c' created
unknown metabolite 'LI_c' created
unknown metabolite '2fe2s_c' created
unknown metabolite '3fe4s_c' created
unknown metabolite 'NiFeCoCN2_c' created
unknown metabolite 'acetyl_c' created


Build all complex formation reactions

TODO: This shouldn't be prining out stuff. Fix them.

In [20]:
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

Created <Metabolite pqq_c at 0x7f55f3240890> in <ComplexFormation formation_GLUCDEHYDROG-MONOMER_mod_pqq at 0x7f55f3240810>
Created <Metabolite hemed_c at 0x7f55f2fe6810> in <ComplexFormation formation_CYT-D-UBIOX-CPLX_mod_pheme_mod_hemed at 0x7f55f2fe6890>
Created <Metabolite EG50003-MONOMER_mod_pan4p at 0x7f55f30116d0> in <ComplexFormation formation_GCVMULTI-CPLX_mod_lipo at 0x7f55f3011650>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_lipo at 0x7f55f3011710> in <ComplexFormation formation_GCVMULTI-CPLX_mod_lipo at 0x7f55f3011650>
Created <Metabolite 23bpg_c at 0x7f55f2a2f590> in <ComplexFormation formation_PHOSGLYCMUTASE_mod_23bpg at 0x7f55f2a2f510>
Created <Metabolite dpm_c at 0x7f55f2968d90> in <ComplexFormation formation_OHMETHYLBILANESYN-MONOMER_mod_dpm at 0x7f55f2968d10>
Created <Metabolite tl_c at 0x7f55f25bcb10> in <ComplexFormation formation_S-ADENMETSYN-CPLX_mod_4:tl_mod_1:generic_divalent at 0x7f55f25bca90>


Crutch reactions for mets that are blocked. TODO remove

## Associate Complexes with Reactions

In [21]:
# associate reaction id with the old ME complex id (including modifications)
rxnToModCplxDict = get_reaction_to_modified_complex(generic=True)

Fixed _DASH:  D__LACtex
Fixed _DASH:  L__LACD2
Fixed _DASH:  L__LACD3
Fixed _DASH:  D__LACt2pp
Fixed _DASH:  L__LACtex
Fixed _DASH:  L__LACt2rpp


In [22]:
for reaction_data in me.stoichiometric_data:
    # Some reactions have no complex associated. Determine if they are
    # spontaneous or orphan reactions.
    try:
        complexes = rxnToModCplxDict[reaction_data.id]
    except KeyError:
        # These are orphans catalyzed by a dummy
        if reaction_data.id == "dummy_reaction" or \
                not rxn_info.is_spontaneous[reaction_data.id]:
            complexes = ["CPLX_dummy"]
        # These are truly spontaneous
        else:
            complexes = [None]
    for complex_id in complexes:
        complex_data = me.complex_data.get_by_id(complex_id) if complex_id else None
        if reaction_data.lower_bound < 0:
            r = MetabolicReaction(reaction_data.id + "_REV_" + str(complex_id))
            me.add_reaction(r)
            r.keff = 65
            r.stoichiometric_data = reaction_data
            if complex_data is not None:
                r.complex_data = complex_data
            r.reverse = True
            r.update(create_new=True)
        if reaction_data.upper_bound > 0:
            r = MetabolicReaction(reaction_data.id + "_FWD_" + str(complex_id))
            me.add_reaction(r)
            r.keff = 65
            r.stoichiometric_data = reaction_data
            if complex_data is not None:
                r.complex_data = complex_data
            r.reverse = False
            r.update(create_new=True)

Created <Metabolite CPLX0-7817_mod_Oxidized at 0x7f55f32c8450> in <MetabolicReaction RNDR1b3_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7f55f32c8410>
Created <Metabolite GLUTAREDOXIN-MONOMER_mod_Oxidized at 0x7f55f32c84d0> in <MetabolicReaction RNDR1b1_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7f55f32c8490>
Created <Metabolite GRXC-MONOMER_mod_Oxidized at 0x7f55f32c8550> in <MetabolicReaction RNDR1b4_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7f55f32c8510>
Created <Metabolite GRXB-MONOMER_mod_Oxidized at 0x7f55f32c86d0> in <MetabolicReaction RNDR1b2_FWD_RIBONUCLEOSIDE-DIP-REDUCTII-CPLX at 0x7f55f32c8690>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_3ha at 0x7f55f32c8d50> in <MetabolicReaction 3OAR401_REV_3-OXOACYL-ACP-REDUCT-MONOMER at 0x7f55f32c8d10>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_act at 0x7f55f32c8d90> in <MetabolicReaction 3OAR401_REV_3-OXOACYL-ACP-REDUCT-MONOMER at 0x7f55f32c8d10>
Created <Metabolite EG50003-MONOMER_mod_pan4p_mod_ddca at 0x7f55f32c8f10> in <Me

In [23]:
# This reaction is weird
me.reactions.get_by_id('CITLY-CPLX_2tpr3dpcoa_FWD_G6340-MONOMER').reaction

'4.27350427350427e-6*mu G6340-MONOMER + 2tpr3dpcoa_c --> '

Rebuild transcription and translation to use tRNA (now that tRNA synthetase complexes are in the model

In [24]:
for r in me.reactions:
    if isinstance(r, tRNAChargingReaction):
        r.update()
for r in me.reactions:
    if isinstance(r, TranslationReaction):
        r.update()
    if isinstance(r, TranscriptionReaction):
        r.update()
    if isinstance(r, MetabolicReaction):
        r.update()

## Add in translocation

In [25]:
translocPath = pandas.read_csv(join(ecoli_files_dir, "translocation_pathways.txt"),sep='\t')

for index, row in translocPath.iterrows():
    translocRxn = ProteinTranslocationData(row.Reaction_name, me)
    translocRxn.keff = float(row.Keff)
    translocRxn.costs_complexes = row.Complexes.split(' AND ')

In [26]:
proteins_sa_coeff_inner={}

transloc = pandas.read_csv(join(ecoli_files_dir, "peptide_compartment_and_pathways2.txt"), sep='\t', comment="#")
for index, row in transloc.iterrows():
    me.metabolites.get_by_id(row.Complex).compartment = row.Complex_compartment
    me.metabolites.get_by_id('protein_'+row.Protein.split('(')[0]).compartment = row.Protein_compartment
    #if index > 316:
    #    continue
    if row.translocase_pathway=='s':
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)
        
        ## This is in preparation for membrane constraint
        mass = me.translation_data.get_by_id(row.Protein.split('(')[0]).mass
        if row.Complex in proteins_sa_coeff_inner.keys():
            proteins_sa_coeff_inner[row.Complex]+=mass*1.21/42.*2
        else:
            proteins_sa_coeff_inner[row.Complex]=mass*1.21/42.*2
    elif row.translocase_pathway == 'r':
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)       
    elif row.translocase_pathway == 'p':
        me.translocation_pathways.srp_yidC_translocation.add_translocation_cost(me,row.Complex,row.Protein)         
    elif row.translocase_pathway =='t':
        me.translocation_pathways.tat_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    elif row.translocase_pathway=='a':
        me.translocation_pathways.srp_translocation.add_translocation_cost(me,row.Complex,row.Protein)
        me.translocation_pathways.secA_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    elif row.translocase_pathway =='l':
        me.translocation_pathways.lol_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    elif row.translocase_pathway =='b':
        me.translocation_pathways.bam_translocation.add_translocation_cost(me,row.Complex,row.Protein)
    elif row.translocase_pathway =='y':
        me.translocation_pathways.yidC_translocation.add_translocation_cost(me,row.Complex,row.Protein)        
    elif row.translocase_pathway!='n':
        print row.translocase_pathway        

sb
pla
sb
sb
sb
pa
sb
sb
pla
pla
sb
sb
sb
pla
pa
sb
sb
sb
pa
sb
sb
sb
sb
sb
sb
sb
pla
pa
sb
sb
sb
pla
sb
sb
pla
sb
pla
pla
pla
pla
sb
pla
ra


Remove unused protein and mRNA to make the model solve faster (TODO remove unused complexes too)

In [27]:
one_rxn_list = []
for met in me.metabolites:
    if len(me.metabolites.get_by_id(met.id.rstrip('_')).reactions) == 1:
        print met, met.reactions
        one_rxn_list.append(met.id)

cpg180_c frozenset([<MetabolicReaction CFAS180G_FWD_CFA-CPLX_mod_hco3 at 0x7f55f34d3d10>])
selnp_c frozenset([<MetabolicReaction SELNPS_FWD_EG10943-MONOMER_mod_generic_divalent at 0x7f55f2b39c90>])
ragund_c frozenset([<MetabolicReaction O16AT_FWD_EG11984-MONOMER at 0x7f55f2ef2310>])
23doguln_c frozenset([<MetabolicReaction DOGULNR_FWD_CPLX0-2061 at 0x7f55f354b510>])
4mhetz_c frozenset([<MetabolicReaction HETZK_FWD_THZ-KIN-MONOMER at 0x7f55f246ac10>])
4hthr_c frozenset([<MetabolicReaction 4HTHRS_FWD_THRESYN-MONOMER_mod_pydx5p at 0x7f55f2b7c110>])
cpe160_c frozenset([<MetabolicReaction CFAS160E_FWD_CFA-CPLX_mod_hco3 at 0x7f55f2f77610>])
gdpfuc_c frozenset([<MetabolicReaction GOFUCR_FWD_CPLX0-3881 at 0x7f55f31cf390>])
cinnm_c frozenset([<MetabolicReaction CINNDO_FWD_HCAMULTI-CPLX at 0x7f55f246a310>])
preq1_c frozenset([<MetabolicReaction CDGR_FWD_QueF_dim at 0x7f55f2e19090>])
hemeO_c frozenset([<MetabolicReaction HEMEOS_FWD_HEMEOSYN-MONOMER_mod_generic_divalent at 0x7f55f3183390>])
aconm_

In [28]:
for met in me.metabolites.query('RNA_b'):
    print met.id, met.reactions

RNA_b0001 frozenset([<TranslationReaction translation_b0001 at 0x7f55f4cd8290>, <Reaction DM_b0001 at 0x7f55f4cd8250>, <TranscriptionReaction transcription_TU00178_from_RpoD_mono at 0x7f55f1252bd0>])
RNA_b0002 frozenset([<Reaction DM_b0002 at 0x7f55f4cd8590>, <TranslationReaction translation_b0002 at 0x7f55f4cd84d0>, <TranscriptionReaction transcription_TU00178_from_RpoD_mono at 0x7f55f1252bd0>])
RNA_b0003 frozenset([<Reaction DM_b0003 at 0x7f55f4cd8510>, <TranscriptionReaction transcription_TU00178_from_RpoD_mono at 0x7f55f1252bd0>, <TranslationReaction translation_b0003 at 0x7f55f4cd8a90>])
RNA_b0004 frozenset([<TranslationReaction translation_b0004 at 0x7f55f4cd8650>, <TranscriptionReaction transcription_TU00178_from_RpoD_mono at 0x7f55f1252bd0>, <Reaction DM_b0004 at 0x7f55f4cd8fd0>])
RNA_b0005 frozenset([<Reaction DM_b0005 at 0x7f55f4cd8410>, <TranscriptionReaction transcription_TU0_12787_from_RpoD_mono at 0x7f55f0af96d0>, <TranslationReaction translation_b0005 at 0x7f55f4cd8f10>]

In [29]:
for c_d in me.complex_data:
    c = c_d.complex
    if len(c.reactions) == 1:
        list(c.reactions)[0].delete(remove_orphans=True)
for p in me.metabolites.query("protein"):
    if len(p._reaction) == 1:
        list(p._reaction)[0].delete(remove_orphans=True)
for m in me.metabolites.query("RNA"):
    if len(m._reaction) == 1:
        list(m._reaction)[0].delete(remove_orphans=True)


This gives the total number of genes included

In [30]:
len(me.reactions.query("transcription"))

5271

In [31]:
len(me.reactions)

18231

In [32]:
len(me.metabolites.query('RNA_'))

4274

In [33]:
len(me.metabolites.query('protein_'))

1439

## Attempt to set keffs

In [34]:
divalent_list = divalent_list
monovalent_list = ['_mod_k','_mod_na1']
from pickle import load
with open("test_keffs.pickle", "rb") as infile:
    old_keffs = load(infile)
keffs = {}

for keff, value in old_keffs.items():
    for i in divalent_list: 
        keff = keff.replace(i, 'generic_divalent')
    for i in monovalent_list: 
        keff = keff.replace(i, '_mod_generic_monovalent')
    keffs[keff] = value
    
for r in me.reactions:
    if isinstance(r, MetabolicReaction) and r.complex_data is None:
        continue
    if isinstance(r, MetabolicReaction) and r.complex_data.id != "CPLX_dummy":
        met_rxn = r
        key = met_rxn.id.replace("-", "_DASH_").replace("__", "_DASH_").replace(":","_COLON_")
        #key = met_rxn.id
        key = "keff_" + key.replace("_FWD_", "_").replace("_REV_", "_")

        matches = [i for i in keffs if key in i]
        # get the direction
        if met_rxn.reverse:
            matches = [i for i in matches if i.endswith("_reverse_priming_keff")]
        else:
            matches = [i for i in matches if i.endswith("_forward_priming_keff")]
        if len(matches) == 1:
            met_rxn.keff = keffs[matches[0]]
        elif len(matches) > 0:
            if len(matches) == len([i for i in matches if key + "_mod_"]):
                met_rxn.keff = keffs[matches[0]]
            else:
                print key, len(matches)
        else:  # len(matches) == 0
            print "no keff found for", key

no keff found for keff_METAT_S_DASH_ADENMETSYN_DASH_CPLX_mod_4_COLON_generic_monovalent_mod_1_COLON_generic_divalent
no keff found for keff_GLUTRR_CPLX0_DASH_3741


## Solve

In [35]:
me.reactions.dummy_reaction_FWD_CPLX_dummy.objective_coefficient = 1.

In [36]:
# Turn off reactions that throw off results as in iOL
KO_list = ['DHPTDNR','DHPTDNRN', 'SUCASPtpp','SUCFUMtpp', 'SUCMALtpp', 'SUCTARTtpp', 
           'CAT', 'FHL', 'SPODM', 'SPODMp']
for reaction in KO_list:
    a = me.reactions.query(reaction + '_')
    for rxn in a:
        rxn.upper_bound = 0
        rxn.lower_bound = 0

In [37]:
me.reactions.EX_glc__D_e.lower_bound = -100

In [ ]:
expressions = compile_expressions(me)

In [ ]:
solve_at_growth_rate(me, 0.1, compiled_expressions=expressions)

In [ ]:
binary_search(me, min_mu=0, max_mu=.1, mu_accuracy=1e-6,
              compiled_expressions=expressions)

In [ ]:
for cplx in generic_RNase_list:
    print me.metabolites.get_by_id(cplx).reactions

In [ ]:
import escher
view = escher.Builder("iJO1366.Central metabolism")
view.reaction_data = me.get_metabolic_flux()
view.display_in_notebook()